<a href="https://colab.research.google.com/github/Gooogr/Book_nlp_with_transformers/blob/main/ch8_optimization_for_production.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this notebook we will explore four complementary opimization techniques for transformer models: 
* Knowledge distillation
* Quantization
* Pruning
* Graph optimization 

with the Open Neural Network Exchange (ONNX) format and ONNX Runtime (ORT).

Let’s suppose that we’re trying to build a text-based assistant for our company’s call
center so that customers can request their account balance or make bookings without
needing to speak with a human agent. In order to understand the goals of a customer,
our assistant will need to be able to classify a wide variety of natural language text
into a set of predefined actions or intents.

In [ ]:
!pip install -qq transformers[sentencepiece] datasets

In [2]:
from transformers import pipeline

In [3]:
bert_ckpt = "transformersbook/bert-base-uncased-finetuned-clinc"
pipe = pipeline("text-classification", model=bert_ckpt)

Downloading:   0%|          | 0.00/7.98k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/418M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/252 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [4]:
query = """Hey, I'd like to rent a vehicle from Nov 1st to Nov 15th in
Paris and I need a 15 passenger van"""
pipe(query)

[{'label': 'car_rental', 'score': 0.5490034818649292}]

In [5]:
pip